In [7]:
import argparse
import sys
import os
import time
import datetime
import ast
from collections import Counter
from pathlib import Path
import hydra

import numpy as np
import torch

# Ensure local src/ is importable
sys.path.insert(0, str(Path(os.getcwd()).parent / "src"))

from my_genQC.inference.eval_metrics import UnitaryFrobeniusNorm, UnitaryInfidelityNorm
from my_genQC.inference.evaluation_helper import get_unitaries, get_srvs
from my_genQC.inference.sampling import generate_compilation_tensors, generate_tensors, decode_tensors_to_backend
from my_genQC.pipeline.diffusion_pipeline import DiffusionPipeline
from my_genQC.platform.simulation import Simulator, CircuitBackendType
from my_genQC.platform.tokenizer.circuits_tokenizer import CircuitTokenizer
from my_genQC.utils.misc_utils import infer_torch_device, get_entanglement_bins
from my_genQC.dataset import circuits_dataset
from my_genQC.models.config_model import ConfigModel
from my_genQC.utils.config_loader import load_config, store_tensor, load_tensor

In [13]:
# Config loading
with hydra.initialize(version_base=None, config_path="../conf"):
    cfg = hydra.compose(config_name="config.yaml", overrides=["evaluation=remote_model"])
    cfg = cfg["evaluation"]

device = torch.device(infer_torch_device())

[INFO]: Cuda device has a capability of 8.6 (>= 8), allowing tf32 matmul.


In [3]:
def load_dataset(dataset_path: Path, device: torch.device):
    config_path = os.path.join(dataset_path, "config.yaml")

    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Config file not found at {config_path}")

    cfg_data = load_config(config_path)
    target = cfg_data.get("target", "")
    if target.endswith("MixedCircuitsConfigDataset"):
        dataset_cls = circuits_dataset.MixedCircuitsConfigDataset
    else:
        dataset_cls = circuits_dataset.CircuitsConfigDataset

    dataset = dataset_cls.from_config_file(
        config_path=config_path,
        device=device,
        save_path=os.path.join(dataset_path, "dataset", "ds")
    )
    
    return dataset

ds = load_dataset("../datasets/paper_quditkit/srv_3q_dataset", device="cuda")

[INFO]: Loading tensor from `../datasets/paper_quditkit/srv_3q_dataset/dataset/ds_x.safetensors` onto device: cuda.
[INFO]: Loading tensor from `../datasets/paper_quditkit/srv_3q_dataset/dataset/ds_y.safetensors` onto device: cuda.
[INFO]: Instantiated config_dataset from given config on cuda.


In [11]:
def load_pipeline(model_dir: Path | None, repo_id: str | None, device: torch.device):
    if repo_id:
        return DiffusionPipeline.from_pretrained(repo_id=repo_id, device=device)

    if not model_dir:
        raise ValueError("Provide either --model-dir or --hf-repo.")

    model_dir = model_dir.resolve()
    config_path = model_dir if model_dir.is_dir() else model_dir.parent
    cfg_file = config_path / "config.yaml"
    if not cfg_file.exists():
        raise FileNotFoundError(f"Missing pipeline config at {cfg_file}")

    return DiffusionPipeline.from_config_file(config_path=str(config_path) + "/", device=device)

In [14]:
pipeline = load_pipeline(
    model_dir=Path(cfg.model_dir) if cfg.model_dir else None,
    repo_id=cfg.hf_repo,
    device=device,
)

pipeline.guidance_sample_mode = "rescaled"
pipeline.scheduler.set_timesteps(cfg.model_params.sample_steps)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

[INFO]: `genQC.models.unet_qc.QC_Cond_UNet` instantiated from given `config` on cuda.
[INFO]: Loading model from `/root/.cache/huggingface/hub/models--Floki00--qc_srv_3to8qubit/snapshots/18eece95eb50483c46dd4eef1cdac3e09afadd54/model.pt` onto device: cuda.
[INFO]: `genQC.models.unet_qc.QC_Cond_UNet`. Freeze model: True


open_clip_model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

[INFO]: `genQC.models.frozen_open_clip.CachedFrozenOpenCLIPEmbedder` instantiated from given `config` on cuda.
[INFO]: `genQC.models.frozen_open_clip.CachedFrozenOpenCLIPEmbedder`. Found no key `save_type` in `config`. No state dict loaded.
[INFO]: `genQC.models.frozen_open_clip.CachedFrozenOpenCLIPEmbedder`. Freeze model: True


In [19]:
pipeline.text_encoder

CachedFrozenOpenCLIPEmbedder(
  (model): CLIP(
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-11): 12 x ResidualAttentionBlock(
          (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=512, out_features=2048, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=2048, out_features=512, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (token_embedding): Embedding(49408, 512)
    (ln_final): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
)

In [22]:
import open_clip
from my_genQC.models.clip.frozen_open_clip import FrozenOpenCLIPEmbedderConfig, FrozenOpenCLIPEmbedder

In [23]:
clip_model = FrozenOpenCLIPEmbedder(
    arch="hf-hub:lysandre/CLIP-ViT-L-14-laion2B-s32B-b82K",
    # version="",
)
clip_model.to(device)

open_clip_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

open_clip_pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

FrozenOpenCLIPEmbedder(
  (model): CLIP(
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-11): 12 x ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (token_embedding): Embedding(49408, 768)
    (ln_final): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
)

In [25]:
pipeline.text_encoder = clip_model